In [1]:
import sys
import os
import numpy as np
import networkx as nx
from sklearn.cluster import KMeans
from collections import Counter

# Silenziare il warning relativo ai core fisici
os.environ["LOKY_MAX_CPU_COUNT"] = "4"  # Imposta il numero di core logici

import sys
project_path = os.getcwd().split("Code")[0]
sys.path.append(project_path)

# Ora puoi importare la classe GraphConstructor
from Code.notebook.graph.GraphConstructor import GraphConstructor

# Funzione per eseguire il clustering con K-Means
def k_means_clustering(graph, num_clusters):
    """
    Esegue il clustering con l'algoritmo K-Means sul grafo.
    
    Args:
        graph (networkx.Graph): Il grafo su cui eseguire il clustering.
        num_clusters (int): Il numero di cluster desiderati.
    
    Returns:
        dict: Un dizionario con l'etichetta di cluster per ciascun nodo.
    """
    # Estrai la matrice di adiacenza come feature
    adjacency_matrix = nx.to_numpy_array(graph)
    
    # Applica K-Means sulla matrice di adiacenza
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(adjacency_matrix)
    
    # Prediciamo le etichette dei cluster
    cluster_labels = kmeans.labels_
    
    # Restituiamo le etichette dei cluster per ciascun nodo
    return {node: label for node, label in zip(graph.nodes, cluster_labels)}

# Funzione per sanitizzare gli attributi del grafo (nodi e archi)
def sanitize_graph_attributes(graph):
    """
    Sanitizza le chiavi e i valori degli attributi dei nodi e degli archi
    per renderli compatibili con il formato GML.
    """
    for node in graph.nodes:
        attrs = graph.nodes[node]
        sanitized_attrs = {}
        for key, value in attrs.items():
            # Rimpiazza spazi e trattini nei nomi delle chiavi
            new_key = str(key).replace(" ", "_").replace("-", "_")
            # Converte tutti i valori in stringhe
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.nodes[node].clear()
        graph.nodes[node].update(sanitized_attrs)

    for u, v, attrs in graph.edges(data=True):
        sanitized_attrs = {}
        for key, value in attrs.items():
            # Rimpiazza spazi e trattini nei nomi delle chiavi
            new_key = str(key).replace(" ", "_").replace("-", "_")
            # Converte tutti i valori in stringhe
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.edges[u, v].clear()
        graph.edges[u, v].update(sanitized_attrs)

# Crea il grafo utilizzando GraphConstructor
graph_builder = GraphConstructor()
graph_builder.build_graph()
graph = graph_builder.graph

# Esegui il clustering con K-Means
num_clusters = 3  # Modifica questo valore a seconda delle tue necessità
kmeans_labels = k_means_clustering(graph, num_clusters)

# Copia il grafo originale per assegnare i cluster
gr = graph.copy()

# Assegna il cluster a ciascun nodo
for node, label in kmeans_labels.items():
    gr.nodes[node]['cluster'] = label  # Assegniamo l'etichetta del cluster al nodo

# Sanitizza gli attributi del grafo prima di esportarlo
sanitize_graph_attributes(gr)

# Esporta il grafo in formato GML
output_path = "graph_with_kmeans_clusters.gml"
nx.write_gml(gr, output_path)

# Visualizza il numero di nodi per cluster
cluster_counts = Counter(kmeans_labels.values())
print(f"Numero di nodi per cluster: {cluster_counts}")

print(f"Clustering con K-Means completato! Grafo esportato in: {output_path}")


Numero di nodi per cluster: Counter({1: 6005, 2: 12, 0: 12})
Clustering con K-Means completato! Grafo esportato in: graph_with_kmeans_clusters.gml
